In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
# return(list(K = ncol(A_hat),
#               pureVec = I_hat,
#               pureInd = I_hat_part,
#               group = group,
#               A = A_hat,
#               C = C_hat,
#               Omega = Omega,
#               Gamma = Gamma_hat,
#               optDelta = optDelta))

Getting latent factors for delta,  0.1 , and lambda,  0.5 . 
f_size is set as  317 
         final marginal spec: 0.1
      starting interaction selection . . . 
[1] "Before doing interaction SLIDE"
[1] "z47"
[1] "printing the yhat of each maginals..."
NULL
      running knockoffs on marginal/interaction submodels...
            only one upsilon . . . using without testing 
[1] "uplsion error cheking: \n"
[1] "ups47"
[1] "upsilon colnames:"
Getting performance for real model... 
Getting performance for fully random model... 
Getting performance for partially random model... 
The number of total LFs is  317 .
The number of standalone LFs is  1 .
The number of interacting LFs is  9 .
The approximation of cross-validation performance is  0.6288715 .


In [4]:
input_params = {
    'x_path' : '/ix3/djishnu/alw399/SLIDE_py/example_data/jing_adult_CTL_KIR_SLE_X.csv',
    'y_path' : '/ix3/djishnu/alw399/SLIDE_py/example_data/jing_adult_CTL_KIR_SLE_y.csv',
    'niter' : 3,
    'spec' : 0.1,
    'fdr' : 1,
    'rep_CV' : 50,
    'pure_homo' : True,
    'lambda' : [0.5],
    'delta' : [0.1],
    'out_path': 'example_results/love',
    'thresh_fdr': 0.2
}

In [9]:
import sys
sys.path.append('src/SLIDE')

In [11]:
from slide import OptimizeSLIDE
slider = OptimizeSLIDE(input_params)

y = slider.data.Y
X = slider.data.X


In [13]:
slider.load_love('example_results/love/0.1_0.5/love_result.pkl')
slider.latent_factors.head(3)

,Z0,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,...,Z288,Z289,Z290,Z291,Z292,Z293,Z294,Z295,Z296,Z297
KIR1_S383,1.275734,2.621241,-1.897056,-3.398389,2.907179,2.781502,0.446547,0.612499,-0.282035,0.623728,...,2.203205,-0.499593,4.403616,2.120190,4.560253,-2.492526,1.594596,0.789552,-0.532972,-0.992736
KIR1_S385,1.103436,0.989648,-2.097103,1.030024,-0.036912,-1.103845,-0.614409,0.365514,-0.709378,1.853581,...,0.410724,-1.601530,1.841387,-0.538300,0.992549,-0.148245,-1.228353,4.355314,-2.337348,-0.795665
KIR1_S388,3.601299,-2.027444,-1.044921,-1.234978,0.856617,-0.095376,-1.857058,-0.360887,2.071314,-3.141949,...,1.118567,4.951853,1.637125,0.146825,-2.154139,-1.901522,-0.132736,-0.830408,0.360119,-3.215163


In [14]:
from knockoffs import Knockoffs

z = slider.latent_factors.values
y = slider.data.Y.values
machop = Knockoffs(y, z)

In [20]:
machop.select_short_freq(z, fdr=0.05, spec=0.2, niter=10, f_size=298, n_workers=1)

Processing subsets: 100%|██████████| 1/1 [00:11<00:00, 11.18s/it]


array([  1,  23,  25,  45,  46,  50,  52,  57,  65,  68,  80,  82, 109,
       117, 119, 120, 129, 131, 137, 150, 156, 157, 158, 187, 207, 231,
       233, 241, 294])

In [76]:
z_scaled = machop.scale_features(z)
y_scaled = machop.y.copy()

In [ ]:
from knockpy import KnockoffFilter

kfilter = KnockoffFilter(
    ksampler='gaussian', 
    fstat='lasso'
)
rejections = kfilter.forward(X=z_scaled, y=y.values, fdr=1, shrinkage="ledoitwolf")
rejections

array([1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
       0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1.,
       1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1.,
       0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
       0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1.,
       0., 1., 1., 1., 0.